In [73]:
import pandas as pd

df_neg = pd.read_csv("data_bersih_negative.csv")
df_pos = pd.read_csv("data_bersih_positive.csv")
df_neg.shape

(195, 3)

In [74]:
import numpy as np

# Menggabungkan DataFrame df_neg dan df_pos
df = pd.concat([df_neg, df_pos], ignore_index=True)

# Mengacak urutan baris dalam DataFrame
shuffled_indices = np.random.permutation(df.index)
df = df.loc[shuffled_indices]
df

,url,text,status
311,https://thinkquest.org,site cant reached thinkquestorg took long resp...,aman
301,https://synonym.com,synonymcom new word condensation synonym anywa...,aman
72,https://cliphunter.com,home categories popular pornstars playlists li...,bahaya
219,https://mentalfloss.com,please note website includes accessibility sys...,aman
78,https://elephanttube.com,bagus news elephanttube merged xxxcom closed p...,bahaya
...,...,...,...
64,https://chaturbate.com,act masturbating chatting online status anonym...,bahaya
331,https://cam.ac.uk,choices little files save device remember pref...,aman
321,https://unt.edu,main find passion unt search enrollment soars ...,aman
26,https://porntube.com,porn tube network tubeatube porner bros fux ho...,bahaya


In [75]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Pra-pemrosesan Teks
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(df['text'].apply(lambda x: np.str_(x)))
y = df['status']

# Pembagian Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pembuatan dan Pelatihan Model SVM
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Evaluasi Model
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Hasil Akurasi : ",accuracy)
print("Laporan Klasifikasi : \n",classification_rep)

Hasil Akurasi :  0.9466666666666667
Laporan Klasifikasi : 
               precision    recall  f1-score   support

        aman       0.90      0.97      0.93        29
      bahaya       0.98      0.93      0.96        46

    accuracy                           0.95        75
   macro avg       0.94      0.95      0.94        75
weighted avg       0.95      0.95      0.95        75



In [76]:
import pickle

# Simpan TF-IDF Vectorizer
with open('Model/tfidf_vectorizer_snailly.pkl', 'wb') as file:
    pickle.dump(tfidf_vectorizer, file)

# Simpan model ke file PKL
pkl_filename = 'Model/model_snaily.pkl'
with open(pkl_filename, 'wb') as pkl_file:
    pickle.dump(svm_model, pkl_file)

In [77]:
import re
import string

# Teks baru yang akan diuji
new_text = "dadang jsn front page site home study materials elementary middle school classes, complete semester natural sciences, write comment, education friends happy learning opportunity time, natural sciences, SDMI accessed directly online used e-learning school home field study iv odd full chapter human skeleton, maintenance function, human function, influence body posture growth"

# Pra-pemrosesan teks baru
# preprocessed_new_text = preprocess_text(new_text)
new_text_vector = tfidf_vectorizer.transform([new_text])

# Memuat kembali model dari file PKL
loaded_svm_model = None
with open('Model/model_snaily_1.pkl', 'rb') as pkl_file:
    loaded_svm_model = pickle.load(pkl_file)

# Gunakan model yang dimuat untuk membuat prediksi
new_predictions = loaded_svm_model.predict(new_text_vector)

# Gunakan model yang dimuat untuk mendapatkan signed distance dari hyperplane
decision_value = loaded_svm_model.decision_function(new_text_vector)

# Menerapkan Platt scaling untuk mengkonversi signed distance menjadi probabilitas
a = loaded_svm_model.coef_[0]
b = loaded_svm_model.intercept_
prob_berbahaya = 1 / (1 + np.exp(-decision_value - b))
prob_aman = 1 - prob_berbahaya

print("Probabilitas Aman:", prob_aman)
print("Probabilitas Berbahaya:", prob_berbahaya)
# Prediksi menggunakan model SVM
# prediction = svm_model.predict(new_text_vector)

print("Prediksi:", new_predictions)

Probabilitas Aman: [0.64116437]
Probabilitas Berbahaya: [0.35883563]
Prediksi: ['aman']
